# Load env and files

In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from fossil_classification import *
from enrich_holdings import *
from reports_etl import *
from holdings_analysis import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.3f}".format

In [4]:
from_year = '2023'
from_q = '4'

In [5]:
response_path = "data/downloaded reports/company reports/" + from_year + "Q" + from_q + "/"
all_holdings_cls = pd.read_csv(response_path+"all_holdings_cls_all_ids.csv", dtype=holdings_dtypes())
# all_holdings_cls = load_mappings_and_add_ids_to_holdings(all_holdings_cls)

/var/folders/rt/89wrjbhs2m5527n7jhtb42jh0000gn/T/ipykernel_14628/222160177.py:2: DtypeWarning: Columns (6,7,9,13,15,20,22,23,24,25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  all_holdings_cls = pd.read_csv(response_path+"all_holdings_cls_all_ids.csv", dtype=holdings_dtypes())


# Let the Analysis begin!

In [6]:
pd.DataFrame(all_holdings_cls.groupby(['SystemName'])['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
SystemName,
ביטוח,"6,563,453,401.04"
גמל,"9,735,230,506.80"
פנסיה,"9,883,981,564.30"


## Get Company level stats over time

### Filter results to include only major companies

In [7]:
all_holdings_cls_filtered = filter_major_companies(all_holdings_cls)
pd.DataFrame(all_holdings_cls_filtered.groupby(['SystemName'])['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
SystemName,
ביטוח,"6,263,588,250.78"
גמל,"8,371,917,078.12"
פנסיה,"9,198,696,070.97"


### * should include other subsidiary funds as well?

In [8]:
# included_comp_group = all_holdings_cls_filtered["ParentCorpGroup"].unique()
# included_comp = all_holdings_cls_filtered["ParentCorpName"].unique()
# all_comp = all_holdings_cls["ParentCorpName"].unique()
# not_included_comp = [c for c in all_comp if c not in included_comp]
# print([c for c in not_included_comp if any(g in re.split('-| ',c) for g in included_comp_group)])

# pd.DataFrame(
#     all_holdings_cls[
#     all_holdings_cls["ParentCorpName"].isin(not_included_comp)
#     ].groupby(["ParentCorpName", "SystemName"])["שווי"].sum()
# ).sort_values("שווי", ascending=False)

In [9]:
company_stats_since_2020 = get_summary(all_holdings_cls_filtered, 'ReportPeriodDate', 'ParentCorpGroup')
company_stats_since_2020 = company_stats_since_2020.sort_values(
    ["ReportPeriodDate", "ParentCorpGroup"], ascending=(False, True)
)

In [10]:
# company_stats_since_2020[company_stats_since_2020['ParentCorpGroup'] == 'מגדל']

In [11]:
company_stats_since_2020.to_csv(response_path+"company_stats_since_2020.csv", index=False)

In [12]:
company_system_stats_since_2020 = get_summary(
    all_holdings_cls_filtered,
    'ReportPeriodDate', 'ParentCorpGroup', 'SystemName'
)

In [13]:
company_system_stats_since_2020.to_csv(response_path+"company_system_stats_since_2020.csv", index=False)

In [14]:
company_system_holding_type_stats_since_2020 = get_summary(
    all_holdings_cls_filtered,
    'ReportPeriodDate', 'ParentCorpGroup', 'SystemName', 'holding_type'
)

In [15]:
company_system_holding_type_stats_since_2020.to_csv(
    response_path+"company_system_holding_type_stats_since_2020.csv", index=False)

### Export aggregated data for quarterly org ranking

In [16]:
midrag_agg = get_latest_q_ranking_agg_from_holdings(filter_major_companies(all_holdings_cls))

In [17]:
midrag_agg.to_csv(
    response_path+"midrag_agg_latest_q.csv", index=False
)

## Mergers - update
### 1. update holdings data according to system mergers

In [18]:
all_holdings_majors = filter_major_companies(all_holdings_cls)
# including subsidiaries
# majors_after_mergers = filter_major_companies(all_holdings_cls, include_subsidiaries=True)

other_quarters = all_holdings_majors[
    all_holdings_majors["ReportPeriodDate"] != '2021-09-30'
#     majors_after_mergers["ReportPeriodDate"].max()
]

# leave only mergers quarter (2021 Q3)
majors_after_mergers = all_holdings_majors[
    all_holdings_majors["ReportPeriodDate"] == '2021-09-30'
#     majors_after_mergers["ReportPeriodDate"].max()
]

# change data according to mergers
majors_after_mergers = update_system_owners(majors_after_mergers, "גמל", "הלמן", "הפניקס")
majors_after_mergers = update_system_owners(majors_after_mergers, "פנסיה", "הלמן", "מיטב")
majors_after_mergers = update_system_owners(majors_after_mergers, "גמל", "פסגות", "אלטשולר")
majors_after_mergers = update_system_owners(majors_after_mergers, "פנסיה", "פסגות", "הראל")

moving 15,677,553.290000001 from הלמן גמל to הפניקס גמל
moving 0.0 from הלמן פנסיה to מיטב פנסיה
moving 49,780,952.0 from פסגות גמל to אלטשולר גמל
moving 3,964,347.4000000004 from פסגות פנסיה to הראל פנסיה


In [19]:
# adding back to other periods
all_holdings_after_mergers = pd.concat([majors_after_mergers, other_quarters])
pd.DataFrame(all_holdings_after_mergers.groupby(['SystemName'])['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
SystemName,
ביטוח,"6,263,588,250.78"
גמל,"8,371,917,078.12"
פנסיה,"9,198,696,070.97"


In [20]:
company_stats_after_mergers = get_summary(
    all_holdings_after_mergers,
    'ReportPeriodDate', 'ParentCorpGroup'
)
# company_system_stats_after_mergers.sort_values("שיעור פוסילי מתוך מניות ואגח סחירים + סוגי החזקות לא פוסיליים")
# company_stats_after_mergers.sort_values("שיעור פוסילי במניות ואגח קונצרני סחירים")[[
#     "ReportPeriodDate",
#     "ParentCorpGroup",
#     "שווי",
#     "שווי במניות ואגח קונצרני סחירים",
#     "שיעור פוסילי במניות ואגח קונצרני סחירים"
# ]]

# company_system_stats_after_mergers["ratio"] = (
#     company_system_stats_after_mergers["שיעור פוסילי במניות ואגח קונצרני סחירים"] / 
#     company_system_stats_after_mergers["שיעור פוסילי מתוך מניות ואגח סחירים + סוגי החזקות לא פוסיליים"]
# )

company_stats_after_mergers.to_csv(response_path+"company_stats_since_2020_after_mergers.csv", index=False)

### Export aggregated data <u> after mergers </u> for quarterly org ranking

In [21]:
midrag_agg_after_mergers = get_latest_q_ranking_agg_from_holdings(all_holdings_after_mergers)

In [22]:
midrag_agg_after_mergers.to_csv(
    response_path+"midrag_after_mergers.csv", index=False
)

Upload results to G-Drive: https://drive.google.com/drive/u/2/folders/1n0F9JpyJ4LdWxLc4-a8XaRaMV20qo1Fx

# Export holdings data for quarterly ranking

In [23]:
q = from_year + ' רבעון ' + from_q
holding_types_for_ranking = ['מניות', 'אג"ח קונצרני']
cols = [
    'שם המנפיק/שם נייר ערך',
    'מספר ני"ע',
    'ISIN',
    'מספר מנפיק',
    'מספר תאגיד',
    'LEI',
    'ענף מסחר',
    'שווי',
    'שעור מנכסי אפיק ההשקעה',
    'שעור מסך נכסי השקעה',
    'שעור מערך נקוב מונפק',
    'holding_type',
    'SystemName',
    'ParentCorpGroup',
    'is_fossil',
    'שווי פוסילי'
]
holdings_for_ranking = filter_major_companies(all_holdings_cls)
holdings_for_ranking = holdings_for_ranking.loc[
    (holdings_for_ranking['ReportPeriodDesc'] == q) &
    (holdings_for_ranking["holding_type"].isin(holding_types_for_ranking)),
    cols
]
# use ISIN when Israeli sec num is not available
holdings_for_ranking['מספר ני"ע'] = holdings_for_ranking['מספר ני"ע'].fillna(holdings_for_ranking['ISIN'])
# use Israeli corp num and then LEI when issuer number is not available
holdings_for_ranking['מספר מנפיק'] = holdings_for_ranking['מספר מנפיק'].fillna(holdings_for_ranking['מספר תאגיד'])
holdings_for_ranking['מספר מנפיק'] = holdings_for_ranking['מספר מנפיק'].fillna(holdings_for_ranking['LEI'])
holdings_for_ranking.drop(['ISIN', 'מספר תאגיד', 'LEI'], axis=1, inplace=True)
holdings_for_ranking.rename({
    "holding_type": "סוג החזקה",
    "SystemName":  "אפיק",
    "ParentCorpGroup": "גוף"
}, axis=1, inplace=True)

# holdings_for_ranking[
#     (holdings_for_ranking["is_fossil"] == 1) &
#     ~(holdings_for_ranking["שווי פוסילי"] > 0)
# ]

holdings_for_ranking.to_csv(response_path+"quarterly_ranking_data.csv", index=False)

## Export csv per institution

In [24]:
inst_cols = [c for c in holdings_for_ranking.columns if (c != 'גוף') & ~(c.startswith("שעור"))]
for inst in get_major_institutions_list():
    holding_per_inst = holdings_for_ranking.loc[
        (holdings_for_ranking["גוף"] == inst),
        inst_cols
    ].sort_values("שווי פוסילי", ascending=False)
    holding_per_inst.to_csv(response_path+inst+".csv", index=False)

# Get a list of all fossil holdings for major institutions for a given quarter

In [25]:
# inst = get_major_institutions_list()
# fossil_holdings_last_q = group_fossil_holdings_quarters_institutions(
#     all_holdings_cls_filtered,
#     quarters=q,
#     institutions = inst
# )

# fossil_holdings_last_q.to_csv("analysis/last_q_major_insts_fossil_holdings.csv", index=False)

<h1><center>*** RUN UP TO HERE ***</center></h1>

# Compare Fossil Holdings over quarters
## 1. for specific companies

In [28]:
# compare fossil holdings

company_for_comparison = ['אלטשולר']
quarters_for_comparison = [
    '2023 רבעון 3',
    '2023 רבעון 4'
]

holding_types = ['מניות', 'אג"ח קונצרני']
# comparison = compare_holdings_over_quarters(all_holdings_cls_filtered, quarters_for_comparison, company_for_comparison)
comparison = compare_holdings_over_quarters(
    all_holdings_after_mergers, holding_types, quarters_for_comparison, company_for_comparison, fossil_only=1)
comparison.sort_values("name", ascending=False)

total Israeli fossil holdings: 5338466.500939828
total fossil holdings with il_sec_num: 5338466.5009398265
total fossil holdings without il_sec_num: 1956071.8206778504
1956071.8206778504
after adding Israeli and non-Israeli: 7294538.321617679


/Users/urimarom/PycharmProjects/fossil_classification/holdings_analysis.py:174: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  selected_holdings = selected_holdings[(holdings["is_fossil"] == 1)]


,name,id,fossil_sum_prev_q,quantity_sum_prev_q,fossil_sum_curr_q,quantity_sum_curr_q,quantity_diff,quantity_diff_pct,fossil_sum_diff,fossil_sum_diff_pct
11,תמר פטרוליום,1689,"149,216.833","153,568,774.260","148,254.641","153,568,774.260",0.000,0.000,-962.192,-0.006
10,קנון,201406588,"292,657.817","3,325,656.990","293,655.512","3,325,657.020",0.030,0.000,997.695,0.003
9,קיסטון ריט,1904,"167,490.073","28,885,755.000","146,681.863","28,885,755.000",0.000,0.000,"-20,808.210",-0.124
8,לפידות,642,"615,779.209","9,284,970.000","522,186.712","9,284,970.000",0.000,0.000,"-93,592.498",-0.152
7,כיל,281,"859,506.343","40,792,897.220","741,614.870","40,792,897.210",-0.010,-0.000,"-117,891.474",-0.137
6,חשמל,600,"13,682.936","11,671,872.000","13,901.200","11,671,872.000",0.000,0.000,218.264,0.016
5,חברה לישראל,576,"219,708.551","226,317.010","206,876.352","226,316.980",-0.030,-0.000,"-12,832.199",-0.058
4,אלקו החזקות,694,"131,615.100","1,168,873.010","137,576.349","1,168,872.970",-0.040,-0.000,"5,961.248",0.045
3,או פי סי אנרגיה,1682,"343,492.009","13,940,422.310","334,570.130","13,940,422.280",-0.030,-0.000,"-8,921.879",-0.026
2,TOTAL SA FP PA,529900S21EQ1BO4ESM68,"406,339.160","1,560,475.990","386,159.988","1,562,676.000","2,200.010",0.001,"-20,179.171",-0.050


In [29]:
company_quarter = company_for_comparison[0] + " - " + quarters_for_comparison[1]
comparison.to_csv("analysis/fossil holdings comparison - " + company_quarter + ".csv", index=False, header=False)

## compare non-fossil holdings - example

In [ ]:
company_for_comparison = ['מגדל']
quarters_for_comparison = [
    '2021 רבעון 4',
    '2022 רבעון 4'
]

holding_types = ['לא סחיר - מניות']
comparison = compare_holdings_over_quarters(
    all_holdings_after_mergers, holding_types, quarters_for_comparison, company_for_comparison, fossil_only=0)
comparison.sort_values("name", ascending=False)

In [ ]:
company_quarter = company_for_comparison[0] + " - " + quarters_for_comparison[1]
holding_types_str = ' '.join(holding_types)
# comparison.to_csv("analysis/holdings comparison - " + company_quarter + " - " + holding_types_str + ".csv", index=False)

In [ ]:
all_holdings_cls_filtered[
    (all_holdings_cls_filtered["ParentCorpGroup"] == 'מגדל') &
    (all_holdings_cls_filtered["holding_type"] == 'לא סחיר - מניות')
].groupby('ReportPeriodDesc')['שווי'].agg('sum')

## add profit/loss per quarter
treating changes in quantity as if they took place at the last day of the quarter:
* If quantity went up (securities were bought): calculate the change for prev_q amount
* If quantity went down (securities were sold): calculate the change for prev_q amount 

previous quarter quantity * (current quarter price - previous quarter price)

In [ ]:
comparison["price_diff"] = (
    (comparison["fossil_sum_curr_q"] / comparison["quantity_sum_curr_q"]) -
    (comparison["fossil_sum_prev_q"] / comparison["quantity_sum_prev_q"])
)

comparison["value_diff"] = comparison["price_diff"] * comparison["quantity_sum_prev_q"]

In [ ]:
print(comparison["value_diff"].sum())
comparison

In [ ]:
# divide to מניות and אגח according to request (ugly but no time...)

In [ ]:
def fossil_profit_per_q_by_holding_type(holdings, quarters, company, holding_types):
    results = []
    for i in range(len(quarters)-1):
        for h in holding_types:
            holdings_filtered_by_type = holdings[holdings["holding_type"] == h]
            quarters_for_comparison = [quarters[i], quarters[i+1]]
            comparison = compare_fossil_holdings_over_quarters(holdings_filtered_by_type, quarters_for_comparison, company)
            comparison["prev_price"] = comparison["fossil_sum_prev_q"] / comparison["quantity_sum_prev_q"]
            comparison["curr_price"] = comparison["fossil_sum_curr_q"] / comparison["quantity_sum_curr_q"]
            comparison["price_diff"] = comparison["curr_price"] - comparison["prev_price"]
            comparison["price_diff_pct"] = comparison["price_diff"] / comparison["prev_price"]
            comparison["value_diff"] = comparison["price_diff"] * comparison["quantity_sum_prev_q"]
            # look into rows with abnormal price diff (might mean something changed in number of shares?)
            print(comparison[
                (comparison["price_diff_pct"] > 0.5) |
                (comparison["price_diff_pct"] < -0.5)
            ])
            profit_end_q = comparison["value_diff"].sum()
            total_end_q = comparison["fossil_sum_curr_q"].sum()
            results.append({
                'quarter': quarters[i+1],
                'holding_type': h,
                'profit': profit_end_q,
                'total': total_end_q
            })
    return pd.DataFrame(results)
            

In [ ]:
quarters = [
    '2020 רבעון 4',
    '2021 רבעון 1',
    '2021 רבעון 2',
    '2021 רבעון 3',
    '2021 רבעון 4',
    '2022 רבעון 1'
]

holding_types = ['מניות', 'אג"ח קונצרני']

fossil_profits_by_type_and_q = fossil_profit_per_q_by_holding_type(all_holdings_after_mergers, quarters, ['מגדל'], holding_types)
fossil_profits_by_type_and_q.to_clipboard(index=False)
fossil_profits_by_type_and_q

In [ ]:
fossil_profits_by_type_and_q.groupby("quarter").sum()

## 2. for all (major) companies

In [ ]:
all_holdings_cls_filtered_grouped = all_holdings_cls_filtered.copy()
all_holdings_cls_filtered_grouped["ParentCorpGroup"] = "כל החברות"
all_major_companies = ["כל החברות"]
quarters_for_comparison = [
    '2021 רבעון 2',
    '2021 רבעון 3'
]

compare_all = compare_fossil_holdings_over_quarters(
    all_holdings_cls_filtered_grouped,
    quarters_for_comparison,
    all_major_companies
)

In [ ]:
quarters = ' '.join(quarters_for_comparison)
compare_all.to_csv("analysis/all companies - fossil holdings comparison - " + quarters + ".csv", index=False)

## Look into non fossil holding types

In [ ]:
all_holdings_cls_non_fossil_types = all_holdings_cls_filtered[
    all_holdings_cls_filtered["holding_type"].isin(get_non_fossil_holding_types())
]

all_holdings_cls_non_fossil_types['holding_type'].unique()

In [ ]:
all_holdings_cls_non_fossil_types.groupby("holding_type").agg({
      "שווי": "sum",
    "שווי פוסילי": "sum"
})

In [ ]:
# holdings that should be non fossil by holding type but are fossil
all_holdings_cls_non_fossil_types[
    all_holdings_cls_non_fossil_types["is_fossil"] == 1
]

## Export holdings with missing classification that should be classified

In [ ]:
missing_cls = all_holdings_cls[
    (all_holdings_cls["holding_type"].isin(['מניות', 'אג"ח קונצרני'])) &
    (all_holdings_cls["is_fossil"].isnull())
]

cols = ['שם המנפיק/שם נייר ערך' ,'מספר ני"ע' ,'מספר מנפיק', 'ISIN', 'מספר תאגיד', 'LEI']
missing_cls_deduped = missing_cls[cols].drop_duplicates()
print(missing_cls_deduped.shape[0])
missing_cls_deduped

In [ ]:
missing_cls_deduped.to_csv(response_path+"missing_cls.csv", index=False)

# Simulation - change fossil classification for selected holdings, analyze results

In [ ]:
# fix data in all_holdings_cls - wrong data for סופרגז
# all_holdings_cls[all_holdings_cls["שם המנפיק/שם נייר ערך"] == 'סופרגז']

In [ ]:
all_holdings_cls.loc[all_holdings_cls["שם המנפיק/שם נייר ערך"] == 'סופרגז',
                    'מספר ני"ע'] = '1166917'

all_holdings_cls.loc[all_holdings_cls["שם המנפיק/שם נייר ערך"] == 'סופרגז',
                     'ISIN'] = 'IL0011669178'

In [ ]:
all_holdings_cls["clean_name"] = all_holdings_cls["שם המנפיק/שם נייר ערך"].apply(clean_company)

In [ ]:
selected_holdings_names = ["כיל", "ICL ", 'איי סי אל', 'איי.סי.אל', 'אייסיאל', 'ISRAEL CHEMICAL', 'כימיקלים לישראל', 'כי"ל',
                           "ICLIT"
#                            "רימון",
#                            "סופרגז", "סופר גז",
#                            "OPC", 'אופיסי', 'או. פי. סי', 'או פי סי',
#                            'חברה לישראל', 'החברה לישראל', 'חברהלישראל','חברה ישראל'
                          ]
name_mask = all_holdings_cls["clean_name"].str.upper().str.startswith(tuple(selected_holdings_names))
all_holdings_cls.loc[name_mask].groupby(['מספר מנפיק'], dropna=False).agg(
    name=pd.NamedAgg(column="clean_name", aggfunc="first")
)

In [ ]:
all_holdings_cls.loc[all_holdings_cls["מספר מנפיק"] != '281'].loc[name_mask].sort_values("שם המנפיק/שם נייר ערך")

In [ ]:
def holdings_col_mask_by_another_mask(holdings, col_name, mask):
    selected_col_values = holdings.loc[mask, col_name].dropna().unique()
    col_mask = holdings[col_name].isin(selected_col_values)
    print("matching holdings by {}: {}".format(col_name, col_mask.sum()))
    return col_mask

In [ ]:
# by issuer num
issuer_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "מספר מנפיק", name_mask)
ISIN_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "ISIN", name_mask)
LEI_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "LEI", name_mask)
il_corp_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "מספר תאגיד", name_mask)
# il_sec_num_mask = holdings_col_mask_by_another_mask(all_holdings_cls, 'מספר ני"ע', name_mask)

In [ ]:
all_ids_mask = name_mask | issuer_mask | ISIN_mask | LEI_mask | il_corp_mask # | il_sec_num_mask
no_name_match = all_holdings_cls[all_ids_mask & (~name_mask)]
print("selected holdings with no name match: {}".format(no_name_match.shape[0]))

In [ ]:
no_name_match.sort_values("is_fossil", ascending=False)

In [ ]:
all_holdings_cls.loc[all_ids_mask, "is_fossil"].value_counts(dropna=False)

In [ ]:
# change classification
all_holdings_cls.loc[all_ids_mask, "is_fossil"] = 0
all_holdings_cls["שווי פוסילי"] = all_holdings_cls["שווי"] * all_holdings_cls["is_fossil"]

## Misc queries

In [ ]:
response_path = "data/downloaded reports/company reports 2022Q1/"
all_holdings_cls = pd.read_csv(response_path+"all_holdings_cls_all_ids.csv", dtype=holdings_dtypes())
# all_holdings_cls = load_mappings_and_add_ids_to_holdings(all_holdings_cls)
current_q = '2022 רבעון 1'

## holding type distribution (asset allocation)

In [ ]:
latest_q_sums_by_htype = all_holdings_cls[
    (all_holdings_cls["ReportPeriodDesc"] == current_q)
].groupby("holding_type")[['שווי']].agg('sum').reset_index()

latest_q_sums_by_htype["pct"] = 100.00 * latest_q_sums_by_htype["שווי"] / latest_q_sums_by_htype["שווי"].sum()

print(latest_q_sums_by_htype["שווי"].sum())
latest_q_sums_by_htype.sort_values('שווי', ascending=False).to_clipboard(index=False)
latest_q_sums_by_htype.sort_values('שווי', ascending=False)

## Non-major companies

In [ ]:
haifa = all_holdings_cls.loc[
    (all_holdings_cls["ParentCorpName"].str.contains("חיפה"))
]

In [ ]:
get_summary(haifa, 'ReportPeriodDesc', 'ParentCorpName')

In [ ]:
haifa[haifa["is_fossil"] == 1].sort_values("שם המנפיק/שם נייר ערך")

In [ ]:
hachshara = all_holdings_cls.loc[
    (all_holdings_cls["ParentCorpName"].str.contains("הכשר"))
]
get_summary(hachshara, 'ReportPeriodDesc', 'ParentCorpName')

## looking into specific holdings
קרנות סל - מגדל 125

In [ ]:
all_holdings_2021q2[
    (all_holdings_2021q2["holding_type"] == 'קרנות סל') &
    (all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains("125")) &
    (all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains("מגדל"))
].groupby('מספר ני"ע')["שווי"].sum()

In [ ]:
# החזקות של חברות בקרן עוקבת תא 125 של מגדל
cols = ["SystemName", "ParentCorpName","שווי", 'שעור מנכסי אפיק ההשקעה', "שעור מערך נקוב מונפק"]
all_holdings_2021q2[
    (all_holdings_2021q2['מספר ני"ע'] == 1150283)
][cols]

In [ ]:
cols = ["שם המנפיק/שם נייר ערך", 'מספר ני"ע', "שווי", "שעור מנכסי אפיק ההשקעה", "שעור מערך נקוב מונפק", "SystemName"]

all_holdings_2021q2[
    (all_holdings_2021q2['holding_type'] == 'קרנות סל') &
    (all_holdings_2021q2["ParentCorpName"].str.contains('מגדל')) &
    (~all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains('כש')) &
    (all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains('125'))
][cols]

## Looking into ETFs

In [ ]:
etf_holdings = all_holdings_2021q2[
    (all_holdings_2021q2["holding_type"] == 'קרנות סל')
]

etf_holdings.groupby("זירת מסחר")["שווי"].sum()

In [ ]:
etf_holdings_tase = etf_holdings[etf_holdings["זירת מסחר"] == "TASE"]
etf_holdings_tase.head()

### Latest quarter holdings per company

In [ ]:
all_holdings_cls_latest_q = all_holdings_cls[
    all_holdings_cls["ReportPeriodDesc"] == '2021 רבעון 3'
]

pd.DataFrame(all_holdings_cls_latest_q.groupby(
    ['ParentCorpName'])['שווי'].agg('sum')
            ).sort_values('שווי', ascending=False)

In [ ]:
all_holdings_cls_filtered.groupby("holding_type")["שווי"].sum().sort_values()

In [ ]:
all_holdings_cls_filtered[
    all_holdings_cls_filtered["holding_type"].str.contains("קרנות השקעה")
    ].head()